<a href="https://colab.research.google.com/github/arsilva02/librarianproject/blob/main/WebScrapingAndFeatureGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [101]:
df = pd.read_csv('data/storygraphExport.csv')

In [102]:
#lets look at the contributers section
df[~df['Contributors'].isna()]['Contributors']

,Contributors
12,Neil Smith (Translator)
15,Frances Riddle (Translator)
17,Lucy Scott (Translator)
20,Jeffrey Angles (Translator)
22,"Stephanie Ortega (Translator), Sophie Haydock ..."
...,...
700,"William Hutson (Contributor), Daveed Diggs (Co..."
712,Cassandra Medcalf (Narrator)
713,Gregory Rabassa (Translator)
714,Carol Brown Janeway


In [103]:
#if a "translator" is found in the contributor section, i will mark it as translated
def translation(df):
    df['Translated'] = df['Contributors'].str.contains(r'\(Translator\)', na=False)
    return df

df = translation(df)
df[df['Translated']]

,Title,Authors,Contributors,ISBN/UID,Format,Read Status,Date Added,Last Date Read,Dates Read,Read Count,...,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Review,Content Warnings,Content Warning Description,Tags,Owned?,Translated
12,Anxious People,Fredrik Backman,Neil Smith (Translator),9781501160837,hardcover,to-read,2025/02/09,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
15,Time of the Flies,Claudia Piñeiro,Frances Riddle (Translator),9781913867867,paperback,to-read,2024/10/20,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
17,On a Woman's Madness,Astrid H. Roemer,Lucy Scott (Translator),9781949641431,hardcover,to-read,2025/02/26,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
20,Killing Kanoko: Selected Poems of Hiromi Ito,Hiromi Itō,Jeffrey Angles (Translator),9780979975547,paperback,to-read,2025/01/08,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"b-phys, translated, challenge, japanese, bought",Yes,True
22,Red House Alley,Else Jerusalem,"Stephanie Ortega (Translator), Sophie Haydock ...",9781915812360,paperback,to-read,2025/01/29,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,Thirst,Marina Yuszczuk,Heather Cleary (Translator),9780593472064,hardcover,to-read,2025/01/10,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
687,Strange Weather in Tokyo,Hiromi Kawakami,Allison Markin Powell (Translator),9781846275081,paperback,to-read,2024/04/03,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"translated, japanese",No,True
698,"1Q84, Vol. 1",Haruki Murakami,Jay Rubin (Translator),NaN,paperback,to-read,2024/01/27,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"translated, japanese",No,True
713,Chronicle of a Death Foretold,Gabriel García Márquez,Gregory Rabassa (Translator),9781400034710,paperback,to-read,2024/02/07,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"translated, spanish",No,True


In [139]:
df[df['Title']=='This Is How You Lose the Time War']

,Title,Authors,Contributors,ISBN/UID,Format,Read Status,Date Added,Last Date Read,Dates Read,Read Count,...,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Review,Content Warnings,Content Warning Description,Tags,Owned?,Translated
567,This Is How You Lose the Time War,"Max Gladstone, Amal El-Mohtar",NaN,9781534430990,paperback,read,2024/04/09,2024/04/12,2024/04/08-2024/04/12,1.0,...,Yes,Yes,Yes,5.0,NaN,NaN,NaN,"non-translated, borrowed, hugo",No,False


In [145]:
test = pd.concat([df[8:10],df[567:568]])

In [125]:
import requests
from bs4 import BeautifulSoup
import math

In [146]:
book_search = test[['Title','Authors','ISBN/UID']]
book_search

,Title,Authors,ISBN/UID
8,The Sun Also Rises,Ernest Hemingway,NaN
9,The End We Start from,Megan Hunter,9780802126894
567,This Is How You Lose the Time War,"Max Gladstone, Amal El-Mohtar",9781534430990


In [153]:
def scraper(df):
  master = 'app.thestorygraph.com'

  for index, row in df.iterrows():
    title = row['Title']
    author = row['Authors'].replace(", ", " ")
    isbn = row['ISBN/UID']

    if pd.isna(isbn):
      search = f"{title.replace(' ', '%20')}%20{author.replace(' ', '%20')}"
    else:
      search = isbn

    print(search)


  print(isbn)

scraper(book_search)

The%20Sun%20Also%20Rises%20Ernest%20Hemingway
9780802126894
9781534430990
9781534430990


In [ ]:

response = requests.get(url)
response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
soup = BeautifulSoup(response.content, "html.parser")

        data = {}